In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error
from IPython.display import Audio, display

In [2]:
fs_orig, signal_orig = wavfile.read('input.wav')

In [3]:
# Stereo to mono if needed
if len(signal_orig.shape) > 1:
    signal_orig = signal_orig[:, 0]

In [4]:
# Normalize
signal_orig = signal_orig / np.max(np.abs(signal_orig))